In [1]:
from flask import Flask,render_template,request,redirect, url_for, flash, Response, json
from flask_wtf import FlaskForm
from wtforms import StringField, IntegerField, PasswordField, SubmitField
from wtforms.validators import ValidationError, DataRequired, Length, EqualTo
from flaskext.mysql import MySQL
import tkinter.messagebox as tkmb
from flask_socketio import SocketIO,emit,send
#import eventlet
#import gevent
#from geventwebsocket import *
#import geventwebsocket

In [2]:
app = Flask(__name__,template_folder='.')
app.config['SECRET_KEY']='LongAndRandomSecretKey'
#print (app.config)
mysql = MySQL()

socketio = SocketIO(app)

app.config['MYSQL_DATABASE_HOST'] = 'localhost'
app.config['MYSQL_DATABASE_USER'] = 'root'
app.config['MYSQL_DATABASE_PASSWORD'] = 'vel0c1ty'
app.config['MYSQL_DATABASE_DB'] = 'user_database'

mysql.init_app(app)


In [3]:
class SignInForm(FlaskForm):
    username = StringField(label=('Enter Your Name:'),validators=[DataRequired(),Length(min=5, max=64, message='Name length must be between %(min)d and %(max)d characters')])
    contact= IntegerField('Contact Number: ',validators = [DataRequired()])
    password = PasswordField('Password: ',validators = [DataRequired(),Length(min=8, message='Password should be at least %(min)d characters long')])
    submit = SubmitField(label=('Submit'))


In [4]:
class SignUpForm(FlaskForm):
    username = StringField(label=('Enter Your Name:'),validators=[DataRequired(),Length(min=5, max=64, message='Name length must be between %(min)d and %(max)d characters')])
    contact= IntegerField('Contact Number: ',validators = [DataRequired()])
    password = PasswordField('Password: ',validators = [DataRequired(),Length(min=8, message='Password should be at least %(min)d characters long')])
    confirm_password = PasswordField ('Confirm Password: ',validators = [DataRequired(message='*Required'), EqualTo('password',message='Passwords should match')])
    submit = SubmitField(label=('Submit'))
    
    def validate_contact (self,contact):
            count = 0
            x = self.contact.data
            while True:
                    x = x//10
                    count += 1
                    if x == 0:
                        break
     
            if count != 10:
                  return Response('Contact number should have 10 digits with no spaces in between')
          #print ('Hello')

    def validate_username (self,username):
        print ('username: ',(self.username.data))
        excluded_char = '!@#$%^&*()'
        for char in self.username.data:
            if (char in excluded_char):
                raise ValidationError (f"Character {char} is not allowed in username.")

In [5]:

@app.route ('/',methods=["GET","POST"])
#@app.route('/signin',methods=["GET","POST"])
@app.route ('/signin/',methods=["GET","POST"])


def test():
    print ('request.args: ',request.args)
    print ('request: ',request)
    #form = SignInForm()
    if request.method == "GET":

        
        print ('Inside if GET: ',request.args)
        if 'username' in request.args :

            conn = mysql.connect()
            cursor = conn.cursor()
        
            try:
                    cursor.execute("SELECT * FROM users WHERE contact = {};".format(request.args['contact']))
                    result = cursor.fetchall()
                    print ('result: ',result)
            
                    if len(result) == 0:
                                message = "Contact number does not exist"
                             #return render_template("signin.html", form = form,message=message)  
                             #tkmb.showerror ('Error',message)
                                print (message)
                                
                                return Response( "Contact number does not exist",status=400)
                    else:
                           for row in result:
                                    print ('row: ', row)
                    
                                    if request.args['username'] == row[0]:
                                          if request.args['password'] == row[2]:
                                                 message = "All OK"
                                                 print (message)
                                                 return Response("All OK",status=200)

                                          else:
                                                message = 'Incorrect password'
                                                #tkmb.showerror ('Error',message)
                                                print (message)
                                                return Response(message,status=400)
                                    else:
                                              message = 'Credentials do not match'
                                     #return render_template("signin.html", form = form,message=message) 
                                              #tkmb.showerror ('Error',message) 
                                              print (message)
                                              return Response(message,status=400)
            except Exception as e:
                          print (e)
            #print ("Could'nt find your account")
    #return render_template("signin.html", form = form)     
            

In [6]:
@app.route('/chatwindow/',methods=["POST"])

def add_friend():
    
    print ("request.args['phone_num2']: ",request.args['phone_num2'])
    print ("request.args['phone_num1']: ",request.args['phone_num1'])
    print ('request: ',request)

    if request.method == "POST":

        
        print ('Inside if POST: ',request.args)
        if ( ('phone_num1' in request.args) or 
            ('phone_num2' in request.args) ):  

            conn = mysql.connect()
            cursor = conn.cursor()
        
            try:     

                    cursor.execute("SELECT * FROM friends WHERE phone_num1 = {};".format(request.args['phone_num1']));
                    pnum1_result = cursor.fetchall()
                    print ('phone_num1 result: ',pnum1_result)
                    
                    
                    cursor.execute("SELECT * FROM friends WHERE phone_num2 = {};".format(request.args['phone_num2']));
                    pnum2_result = cursor.fetchall()
                    print ('phone_num2 result: ',pnum2_result)
                    
                    
                    if ( (len(pnum1_result) != 0) and (len(pnum2_result) != 0) ):        
    
                            message = "Contact number already in your friend's list"
                            return Response( message,status=200)
                             
                    else:
                              try:
                                   
                                        cursor.execute ("INSERT INTO friends (phone_num1,phone_num2,friend_name) VALUES (%s, %s, %s)", 
                                                       (request.args['phone_num1'],
                                                        request.args['phone_num2'],
                                                        request.args['friend_name']));

                                        conn.commit()
                                        insert_result = cursor.fetchall()
                                        print ('insert_result: ',insert_result)
                                        message = "Contact added to your friend's list"
                                        return Response(message,status=200)
                                    
                              except Exception as e:
                                        print (e)
                                        return Response(e,status=400)
                                    
               
            except Exception as e:
                    print (e)
                    return Response(e,status=400)
    
    return Response('Error',status=500)


In [7]:
@app.route('/sendmessage/',methods=["POST"])

def send_msg():
    
    print ("request.args['P2']: ",request.args['P2'])
    print ("request.args['P1']: ",request.args['P1'])
    print ('request: ',request)

    if request.method == "POST":
        print ('Inside if POST: ',request.args)
        if ( ('P1' in request.args) or 
            ('P2' in request.args) ):  
            conn = mysql.connect()
            cursor = conn.cursor()
            try:  
                cursor.execute ("INSERT INTO chats (P1,sender,P2,receiver,message) VALUES (%s,%s,%s,%s,%s)" ,
                                (request.args['P1'], 
                                 request.args['sender'],
                                 request.args['P2'],
                                 request.args['receiver'],
                                 request.args['message'])
                                 );
                conn.commit()
                
                
   
                message = "Message updated in table"
                return Response(message,status=200)

            except Exception as e:
                    print (e)
                    return Response(e,status=400)
    
    return Response('Error',status=500)


In [8]:
@socketio.on('connect')
def test_connect():
    print ('Client ' + request.sid +' connected to server')
    socketio.emit('assign_client_id',data=request.sid, room=request.sid)
  

In [9]:
@socketio.on('msg_to_frnd')
def one_to_one_msg(incoming_msg_dict):
    conn = mysql.connect()
    cursor = conn.cursor()
    try:  
        cursor.execute ("INSERT INTO chats (P1,sender,P2,receiver,message) VALUES (%s,%s,%s,%s,%s)" ,
                                (incoming_msg_dict['P1'], 
                                 incoming_msg_dict['sender'],
                                 incoming_msg_dict['P2'],
                                 incoming_msg_dict['receiver'],
                                 incoming_msg_dict['message'])
                                 );
        conn.commit()
        message = "Message updated in table"
        status="OK"
        print ('status: ',status)
        socketio.emit('chat_table_update_resp', data = {'message': message,'status':status})
        
        try:
                    print ('SELECT * FROM chats WHERE id=(SELECT MAX(id) FROM chats) ')
                    cursor.execute("SELECT * FROM chats WHERE id=(SELECT MAX(id) FROM chats);")
                    result = cursor.fetchall()   
                    if len(result) != 0:
                                 #response = jsonify({'error': 'forbidden', 'message': message})
                                 #response.status_code = 403
                                 #return response
                                    socketio.emit('latest_msg', data=json.dumps(result),broadcast=True)   
                                

                    else:
                        message = "No latest records found"
                        status = "Not OK"
                        socketio.emit('errors', data = {'message': message,'status':status})  
                   
        except Exception as e:
                    print (e)
                    message = "Not able to fetch latest records"
                    status = "Not OK"
                    socketio.emit('errors', data = {'message': message,'status':status})  

        #socketio.emit('msg_from_frnd', data=incoming_msg_dict['message'])
        
    except Exception as e:
        print (e)
        message = "Error updating chat table"
        status="Not OK"
        print ('status: ',status)
        socketio.emit('chat_table_update_resp', data = {'message': message,'status':status})            
    
    #socketio.emit('msg_from_frnd', data=incoming_data['message'])


                

In [10]:
@app.route('/msglog/',methods=["GET"])

def msg_log():
    
    print ("request.args['P2']: ",request.args['P2'])
    print ("request.args['P1']: ",request.args['P1'])
    print ('request: ',request)

    if request.method == "GET":
        print ('Inside msglog if GET: ',request.args)
        if ( ('P1' in request.args) and 
            ('P2' in request.args) ):  
            conn = mysql.connect()
            cursor = conn.cursor()
            
            try:
                    cursor.execute("""SELECT * 
                                      FROM chats 
                                      WHERE (P1={} AND P2={})
                                      OR    (P1={} AND P2={});""".format(request.args['P1'],request.args['P2'],request.args['P2'],request.args['P1']))
                    result = cursor.fetchall()   
                    if len(result) != 0:
                                 #response = jsonify({'error': 'forbidden', 'message': message})
                                 #response.status_code = 403
                                 #return response
                                
                                 response = app.response_class(
                                               response=json.dumps(result),
                                               status=200,
                                               mimetype='application/json'
                                               )
                                 return response
                    else:
                        message = "No records found"
                        return Response(message,status=200)
                   
            except Exception as e:
                    print (e)
                    return Response(e,status=400)

    
    return Response('Error',status=500)



In [11]:
@app.route ('/displaylist/',methods=["GET"])


def display_list():
    if request.method == "GET":
            if 'phone_num1' in request.args:
                conn = mysql.connect()
                cursor = conn.cursor()
           
                try:     

                    cursor.execute("SELECT friend_name, phone_num2 FROM friends WHERE phone_num1 = {};".format(request.args['phone_num1']));
                    result = cursor.fetchall()
        
                    print ('friends list result: ',result)
            
                    if len(result) != 0:
                                 #response = jsonify({'error': 'forbidden', 'message': message})
                                 #response.status_code = 403
                                 #return response
                                
                                 response = app.response_class(
                                               response=json.dumps(result),
                                               status=200,
                                               mimetype='application/json'
                                               )
                                 return response
                            
                except Exception as e:
                    print (e)
                    return Response(e,status=400) 
    return Response('Error',status=500)          

In [12]:
@app.route ('/signup/',methods=["GET","POST"])

def index():
     form = SignUpForm()
    #print ('request: ', request.form)
     if request.method == "POST": 
        if form.validate_on_submit():
                details = request.form
                input_username = details['username']
                input_contact = details['contact']
                input_password = details['password']
                input_confirm_password = details['confirm_password']
                
            
                #cur = mysql.get_db().cursor()
                conn = mysql.connect()
                cursor = conn.cursor()
                try:
                    
                        cursor.execute("INSERT INTO users(username, contact, password, confirm_password) VALUES (%s, %s, %s, %s)", 
                            (input_username, input_contact, input_password, input_confirm_password))
                        conn.commit()
                        flash ('User successfully registered, please login with your name and password')
                        #return redirect('/')
                        return render_template("signin.html", form = form)
                except Exception as e:
                       if e.args[0] == 1062:
                            message = 'User already exists'
                            return render_template("index.html", form = form,message= message)
                
                #mysql.connect().close()
                #return 'success'
          
        
        
     return render_template("index.html", form = form)
        
        
        
        
        #print ('is_submitted: ', form.is_submitted())
        #print ('form.validate: ', form.validate())
        #print ('form.validate_on_submit(): ',form.validate_on_submit())
        #return f'''<h1> Welcome {form.username.data} </h1>'''
    #print ('errors: ', form.errors)
    #print ('items: ',form.errors.items())
    #return render_template('index.html', form=form)
    #return None

In [ ]:
if __name__ == '__main__':
    socketio.run(app, host = '127.0.0.1', port=5000)

[2021-10-11 10:49:14,139] WARNING in __init__: WebSocket transport not available. Install gevent-websocket for improved performance.


request.args:  ImmutableMultiDict([('username', 'Susmitha'), ('contact', '9704056271'), ('password', 'myhome12')])
request:  <Request 'http://127.0.0.1:5000/signin/?username=Susmitha&contact=9704056271&password=myhome12' [GET]>
Inside if GET:  ImmutableMultiDict([('username', 'Susmitha'), ('contact', '9704056271'), ('password', 'myhome12')])
result:  (('Susmitha', '9704056271', 'myhome12', 'myhome12'),)
row:  ('Susmitha', '9704056271', 'myhome12', 'myhome12')
All OK
friends list result:  (('Aishu', '1234567890'), ('NAjay', '1234567899'), ('NSivaP', '7702954705'))


The WebSocket transport is not available, you must install a WebSocket server that is compatible with your async mode to enable it. See the documentation for details. (further occurrences of this error will be logged with level INFO)


request.args['P2']:  7702954705
request.args['P1']:  9704056271
request:  <Request 'http://127.0.0.1:5000/msglog/?P1=9704056271&P2=7702954705' [GET]>
Inside msglog if GET:  ImmutableMultiDict([('P1', '9704056271'), ('P2', '7702954705')])
Client KVayc0I62jbW-70ZAAAB connected to server
request.args:  ImmutableMultiDict([('username', 'NSivaP'), ('contact', '7702954705'), ('password', 'myhome02')])
request:  <Request 'http://127.0.0.1:5000/signin/?username=NSivaP&contact=7702954705&password=myhome02' [GET]>
Inside if GET:  ImmutableMultiDict([('username', 'NSivaP'), ('contact', '7702954705'), ('password', 'myhome02')])
result:  (('NSivaP', '7702954705', 'myhome02', 'myhome02'),)
row:  ('NSivaP', '7702954705', 'myhome02', 'myhome02')
All OK
friends list result:  (('HomeMinister', '9704056271'),)
request.args['P2']:  9704056271
request.args['P1']:  7702954705
request:  <Request 'http://127.0.0.1:5000/msglog/?P1=7702954705&P2=9704056271' [GET]>
Inside msglog if GET:  ImmutableMultiDict([('P1

TypeError: connect() missing 1 required positional argument: 'self'

NameError: name 'python' is not defined

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

In [6]:
dir (MySQL)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'connect',
 'get_db',
 'init_app',
 'teardown_request']

In [6]:
MySQL.connect().Error?

Object `Error` not found.


In [ ]:
MySQL.connect().Error

In [7]:
dir(MySQL.connect().Error)

TypeError: connect() missing 1 required positional argument: 'self'

In [7]:
dir(request)

[]

In [8]:
request.

<LocalProxy unbound>

In [1]:
1+2

3